In [1]:
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

In [2]:
test_tensor = np.random.rand(5000).reshape(10, 10, 10, 5)

In [8]:
test_tensor = tf.random.uniform(
    shape=(10, 100, 100, 5), minval=0, maxval=1
)

In [12]:
len(test_tensor.shape)

4

In [13]:
t1 = tf.random.uniform(shape=(10, 10, 3), minval=0, maxval=1)
t2 = tf.random.uniform(shape=(10, 10, 1), minval=0, maxval=1)

In [15]:
tf.concat([t1, t2], -1)

<tf.Tensor: shape=(10, 10, 4), dtype=float32, numpy=
array([[[0.0587604 , 0.12754643, 0.8968291 , 0.10323155],
        [0.86703694, 0.37630415, 0.71292317, 0.8560263 ],
        [0.39672887, 0.5107013 , 0.3717903 , 0.7707652 ],
        [0.48079276, 0.45359576, 0.29256952, 0.91703665],
        [0.7902434 , 0.9169401 , 0.28108573, 0.7052361 ],
        [0.9167485 , 0.2994473 , 0.42821527, 0.8984984 ],
        [0.32482433, 0.71577835, 0.21071517, 0.9541323 ],
        [0.15331006, 0.8363286 , 0.40028656, 0.08530498],
        [0.38325143, 0.00656676, 0.26541126, 0.36438572],
        [0.93045413, 0.00747716, 0.14337444, 0.9266237 ]],

       [[0.82805526, 0.39371848, 0.6846391 , 0.06310523],
        [0.6493143 , 0.7433791 , 0.82159376, 0.49688458],
        [0.00619829, 0.90502477, 0.88169   , 0.07785439],
        [0.24039257, 0.32461357, 0.06886399, 0.38995397],
        [0.5400109 , 0.8874103 , 0.4083383 , 0.39527535],
        [0.64379513, 0.8511015 , 0.4754455 , 0.3412702 ],
        [0.907383